In [ ]:
import pandas as pd
import numpy as np
import pyproj
from shapely.geometry import box, Point
import geopandas as gpd

# 假设你有车辆数据和网格中心点数据
# df_vehicle 包含车辆的经纬度和时间戳信息
# df_grid 包含网格的中心点经纬度信息

# 示例数据
# df_vehicle: 车辆数据，包含 vehicle_id, timestamp, latitude, longitude
# df_grid: 网格中心点数据，包含 grid_id, grid_latitude, grid_longitude

# 1. 将经纬度转为投影坐标系 (UTM)
proj = pyproj.Proj(proj='utm', zone=XX, ellps='WGS84')  # 替换 XX 为你所在区域的 UTM 投影带

# 将车辆的经纬度转换为投影坐标
df_vehicle['x'], df_vehicle['y'] = proj(df_vehicle['longitude'].values, df_vehicle['latitude'].values)
# 将网格的经纬度转换为投影坐标
df_grid['x'], df_grid['y'] = proj(df_grid['grid_longitude'].values, df_grid['grid_latitude'].values)

# 2. 生成固定大小的网格 (20米 x 20米)
def create_grid(df_grid, grid_size=20):
    polygons = []
    for index, row in df_grid.iterrows():
        half_size = grid_size / 2
        polygons.append(box(row['x'] - half_size, row['y'] - half_size, row['x'] + half_size, row['y'] + half_size))
    return gpd.GeoDataFrame(df_grid, geometry=polygons)

# 创建网格的 GeoDataFrame
gdf_grid = create_grid(df_grid)

# 3. 将车辆位置映射到网格
# 计算网格最小的 x 和 y 坐标，用于计算相对的网格索引
x_min = gdf_grid['x'].min()
y_min = gdf_grid['y'].min()
grid_size = 20  # 20 米网格

# 将车辆坐标映射到网格编号
df_vehicle['grid_x_index'] = np.floor((df_vehicle['x'] - x_min) / grid_size).astype(int)
df_vehicle['grid_y_index'] = np.floor((df_vehicle['y'] - y_min) / grid_size).astype(int)

# 创建唯一的网格ID (如: grid_x_index 和 grid_y_index 的组合)
df_vehicle['grid_id'] = df_vehicle['grid_x_index'].astype(str) + '_' + df_vehicle['grid_y_index'].astype(str)

# 4. 按小时统计每个网格的车辆数
df_vehicle['timestamp'] = pd.to_datetime(df_vehicle['timestamp'])
df_vehicle['hour'] = df_vehicle['timestamp'].dt.floor('H')  # 按小时取整

# 按小时和网格ID统计车辆数量
grid_heatmap = df_vehicle.groupby(['hour', 'grid_id']).size().reset_index(name='vehicle_count')

# 输出结果
print(grid_heatmap)
